# Reconocimiento de Entidades Nombradas
## Ejercicio Práctico - Procesamiento de Lenguaje Natural

**Objetivos de Aprendizaje:**
1. Implementar NER usando modelos pre-entrenados en español
2. Crear interfaces interactivas con Gradio
3. Comparar enfoques: Transformers vs API Gemini
4. Desarrollar prototipos rápidos para aplicaciones de PLN

---
**Entorno recomendado:** Google Colab o Amazon SageMaker Studio

**Tiempo estimado:** 60-90 minutos

## Instalación de Dependencias

In [1]:
# Instalación de librerías necesarias
%%capture
!pip install -q transformers torch gradio google-genai

In [2]:
# Verificar instalación
import sys
print(f"Python: {sys.version}")
print("Todas las dependencias instaladas correctamente")

Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Todas las dependencias instaladas correctamente


## Configuración de APIs

### Para Google Colab:
1. Andá a la barra lateral izquierda y hacé clic en 🔑 (Secrets)
2. Agrega una nueva clave: `GOOGLE_API_KEY`
3. Pega tu API key de Google AI Studio

### Para SageMaker Studio:
1. Configura las variables de entorno en tu instancia
2. O usa el método de input manual más abajo

In [17]:
import os
import warnings
warnings.filterwarnings('ignore')

# Configuración de API Key para Gemini
try:
    # Método 1: Google Colab Secrets
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("API Key cargada desde Google Colab Secrets")
except:
    # Método 2: Variables de entorno (SageMaker)
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY:
        print("API Key cargada desde variables de entorno")
    else:
        print("No se encontró GOOGLE_API_KEY")
        print("Podes continuar solo con la parte de Transformers")

API Key cargada desde Google Colab Secrets


---
# PARTE 1: NER con Transformers de Hugging Face

Utilizaremos un modelo especializado en español que puede identificar:
- **PER**: Personas
- **LOC**: Lugares
- **ORG**: Organizaciones  
- **MISC**: Misceláneo

In [ ]:
from transformers import pipeline
import torch

# Verificar disponibilidad de GPU
device = 0 if torch.cuda.is_available() else -1
print(f"🖥️  Dispositivo: {'GPU' if device == 0 else 'CPU'}")

# Cargar modelo de NER en español
print("📥 Cargando modelo de NER para español...")
MODEL_NAME = "mrm8488/bert-spanish-cased-finetuned-ner"

try:
    ner_pipeline = pipeline(
        "ner",
        model=MODEL_NAME,
        aggregation_strategy="simple",
        device=device
    )
    print(f"Modelo {MODEL_NAME} cargado exitosamente")
except Exception as e:
    print(f"❌ Error al cargar modelo: {e}")
    ner_pipeline = None

In [5]:
# Texto de ejemplo con contexto argentino
texto_ejemplo = """
Hola, soy María González y trabajo en la Universidad de Buenos Aires.
Vivo en el barrio de San Telmo y mi empresa favorita es MercadoLibre.
La semana pasada visité el Obelisco con mi amigo Carlos Pérez,
quien trabaja en Google Argentina. Nos encontramos en la estación
Constitución del subte y fuimos a comer un asado en La Boca.
"""

In [6]:
def analizar_entidades_transformers(texto):
    """Procesa texto y extrae entidades usando Transformers"""
    if not ner_pipeline:
        return []

    entidades = ner_pipeline(texto)

    # Formatear resultados
    resultados = []
    for ent in entidades:
        resultados.append({
            'texto': ent['word'],
            'etiqueta': ent['entity_group'],
            'confianza': round(ent['score'], 3),
            'posicion': (ent['start'], ent['end'])
        })

    return resultados

In [7]:
# Probar el modelo
print("🔍 Analizando texto de ejemplo...")
print(f"📝 Texto: {texto_ejemplo.strip()}")
print("\n📊 Entidades encontradas:")

entidades_encontradas = analizar_entidades_transformers(texto_ejemplo)
for ent in entidades_encontradas:
    print(f"  • {ent['texto']} → {ent['etiqueta']} (confianza: {ent['confianza']})")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🔍 Analizando texto de ejemplo...
📝 Texto: Hola, soy María González y trabajo en la Universidad de Buenos Aires.
Vivo en el barrio de San Telmo y mi empresa favorita es MercadoLibre.
La semana pasada visité el Obelisco con mi amigo Carlos Pérez,
quien trabaja en Google Argentina. Nos encontramos en la estación
Constitución del subte y fuimos a comer un asado en La Boca.

📊 Entidades encontradas:
  • María González → PER (confianza: 0.9990000128746033)
  • Universidad de Buenos Aires → ORG (confianza: 0.9990000128746033)
  • San Telmo → LOC (confianza: 0.9980000257492065)
  • MercadoLibre → ORG (confianza: 0.996999979019165)
  • Obelisco → LOC (confianza: 0.9959999918937683)
  • Carlos Pérez → PER (confianza: 1.0)
  • Google Argentina → ORG (confianza: 0.9860000014305115)
  • Constitución → LOC (confianza: 0.9950000047683716)
  • La Boca → LOC (confianza: 1.0)


---
# PARTE 2: NER con API de Gemini

Utilizaremos la API de Gemini para un análisis más detallado y contextual.

In [24]:
# Configurar cliente Gemini
cliente_gemini = None

if GOOGLE_API_KEY:
    try:
        from google import genai
        cliente_gemini = genai.Client(api_key=GOOGLE_API_KEY)
        print("Cliente Gemini configurado correctamente")
    except Exception as e:
        print(f"Error al configurar Gemini: {e}")
else:
    print("API Key de Gemini no disponible")
    print("Podes obtener una gratis en: https://ai.google.dev/")

Cliente Gemini configurado correctamente


In [25]:
def analizar_entidades_gemini(texto):
    """Analiza entidades usando Gemini API"""
    if not cliente_gemini:
        return "❌ Cliente Gemini no disponible"

    prompt = f"""
    Extraé todas las entidades nombradas del siguiente texto en español argentino y clasificálas:

    CATEGORÍAS:
    - PERSONA: Nombres de personas
    - LUGAR: Ciudades, países, barrios, direcciones, lugares específicos
    - ORGANIZACIÓN: Empresas, universidades, instituciones
    - MISCELÁNEO: Otros nombres propios (productos, eventos, marcas)

    FORMATO DE RESPUESTA:
    [ENTIDAD] → [CATEGORÍA] → [BREVE EXPLICACIÓN]

    TEXTO A ANALIZAR:
    {texto}
    """

    try:
        respuesta = cliente_gemini.models.generate_content(
            model="gemini-2.0-flash",
            contents=[prompt]
        )
        return respuesta.text
    except Exception as e:
        return f"❌ Error: {e}"

# Probar Gemini si está disponible
if cliente_gemini:
    print("🔍 Analizando con Gemini...")
    resultado_gemini = analizar_entidades_gemini(texto_ejemplo)
    print("\nAnálisis de Gemini:")
    print(resultado_gemini)
else:
    print("⏭️  Saltando análisis con Gemini (API Key no disponible)")

🔍 Analizando con Gemini...

Análisis de Gemini:
*   María González → PERSONA → Nombre de la persona que habla.
*   Universidad de Buenos Aires → ORGANIZACIÓN → Nombre de la universidad donde trabaja.
*   San Telmo → LUGAR → Nombre del barrio donde vive.
*   MercadoLibre → ORGANIZACIÓN → Empresa favorita de la persona.
*   Obelisco → LUGAR → Monumento de la Ciudad de Buenos Aires.
*   Carlos Pérez → PERSONA → Nombre del amigo de la persona.
*   Google Argentina → ORGANIZACIÓN → Filial argentina de la empresa Google.
*   Constitución → LUGAR → Nombre de la estación de subte.
*   La Boca → LUGAR → Barrio de la Ciudad de Buenos Aires.



---
# PARTE 3: Interfaces Interactivas con Gradio

Crearemos interfaces web interactivas para probar nuestros modelos.

In [26]:
import gradio as gr

def interfaz_ner_transformers(texto):
    """Interfaz para el modelo de Transformers"""
    if not texto.strip():
        return {"text": "Ingresa un texto para analizar", "entities": []}

    if not ner_pipeline:
        return {"text": "Modelo no disponible", "entities": []}

    # Procesar con Transformers
    entidades = ner_pipeline(texto)

    # Formatear para Gradio HighlightedText
    entidades_gradio = []
    for ent in entidades:
        entidades_gradio.append({
            "entity": ent["entity_group"],
            "word": ent["word"],
            "start": ent["start"],
            "end": ent["end"],
            "score": ent["score"]
        })

    return {"text": texto, "entities": entidades_gradio}

# Ejemplos para la interfaz
ejemplos_arg = [
    "Me llamo Juan Pérez y trabajo en el Banco Nación en Buenos Aires.",
    "Cristina Kirchner fue presidenta de Argentina y vive en Santa Cruz.",
    "River Plate jugará contra Boca Juniors en el estadio Monumental.",
    "Lionel Messi nació en Rosario y jugó en el Barcelona.",
    "La Universidad de La Plata es muy prestigiosa en Argentina."
]

# Crear interfaz
demo_transformers = gr.Interface(
    fn=interfaz_ner_transformers,
    inputs=[
        gr.Textbox(
            label="📝 Texto a analizar",
            placeholder="Escribe aquí tu texto en español...",
            lines=4
        )
    ],
    outputs=[
        gr.HighlightedText(
            label="🎯 Entidades Identificadas",
            show_legend=True
        )
    ],
    title="NER con Transformers - Español (de argentina)",
    description="""
    **Modelo:** `mrm8488/bert-spanish-cased-finetuned-ner`

    Identifica entidades nombradas en textos en español:
    - 🧑 **PER**: Personas
    - 🌍 **LOC**: Lugares
    - 🏢 **ORG**: Organizaciones
    - 📦 **MISC**: Misceláneo
    """,
    examples=ejemplos_arg,
    allow_flagging="never",
    theme=gr.themes.Soft()
)

print("✅ Interfaz de Transformers creada")

✅ Interfaz de Transformers creada


In [27]:
# Interfaz para Gemini (solo si está disponible)
if cliente_gemini:
    def interfaz_ner_gemini(texto):
        """Interfaz para Gemini API"""
        if not texto.strip():
            return "Ingresa un texto para analizar"
        return analizar_entidades_gemini(texto)

    demo_gemini = gr.Interface(
        fn=interfaz_ner_gemini,
        inputs=[
            gr.Textbox(
                label="📝 Texto a analizar",
                placeholder="Escribe aquí tu texto en español...",
                lines=4
            )
        ],
        outputs=[
            gr.Textbox(
                label="🧠 Análisis de Gemini",
                lines=10
            )
        ],
        title="NER con Gemini - Análisis Detallado",
        description="""
        **Modelo:** Google Gemini 2.0 Flash

        Análisis avanzado de entidades nombradas con explicaciones contextuales
        optimizado para español argentino.
        """,
        examples=ejemplos_arg,
        allow_flagging="never",
        theme=gr.themes.Soft()
    )
    print("✅ Interfaz de Gemini creada")
else:
    print("⏭️  Interfaz de Gemini no creada (API Key no disponible)")

✅ Interfaz de Gemini creada


In [28]:
# Interfaz comparativa (solo si ambos están disponibles)
if ner_pipeline and cliente_gemini:
    def comparar_modelos(texto):
        """Compara resultados de ambos modelos"""
        if not texto.strip():
            return "Ingresa texto para comparar", "Ingresa texto para comparar"

        # Resultado Transformers
        entidades_tf = analizar_entidades_transformers(texto)
        resultado_tf = "TRANSFORMERS:\n\n"
        for ent in entidades_tf:
            resultado_tf += f"• {ent['texto']} → {ent['etiqueta']} (confianza: {ent['confianza']})\n"

        # Resultado Gemini
        resultado_gemini = "GEMINI:\n\n" + analizar_entidades_gemini(texto)

        return resultado_tf, resultado_gemini

    demo_comparativo = gr.Interface(
        fn=comparar_modelos,
        inputs=[
            gr.Textbox(
                label="📝 Texto a comparar",
                placeholder="Ingresa texto para ver la comparación...",
                lines=3
            )
        ],
        outputs=[
            gr.Textbox(label="Transformers", lines=8),
            gr.Textbox(label="Gemini", lines=8)
        ],
        title="⚔️ Comparación: Transformers vs Gemini",
        description="Compara los resultados de ambos enfoques lado a lado.",
        examples=[
            "Diego Maradona jugó en Boca Juniors y en el Napoli de Italia.",
            "El gobierno argentino anunció medidas desde Casa Rosada."
        ],
        allow_flagging="never"
    )
    print("✅ Interfaz comparativa creada")
else:
    print("⏭️  Interfaz comparativa no creada (requiere ambos modelos)")

✅ Interfaz comparativa creada


## 🚀 Lanzar Interfaces

Ejecuta las celdas siguientes para lanzar las interfaces interactivas:

In [29]:
# Lanzar interfaz de Transformers
if ner_pipeline:
    print("🚀 Lanzando interfaz de Transformers...")
    demo_transformers.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: modelo de Transformers no disponible")

🚀 Lanzando interfaz de Transformers...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8ff6be46dc625d3d35.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [30]:
# Lanzar interfaz de Gemini
if cliente_gemini:
    print("🚀 Lanzando interfaz de Gemini...")
    demo_gemini.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: API de Gemini no disponible")

🚀 Lanzando interfaz de Gemini...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9b8c08c2a919dfdafc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [31]:
# Lanzar interfaz comparativa
if ner_pipeline and cliente_gemini:
    print("🚀 Lanzando interfaz comparativa...")
    demo_comparativo.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: requiere ambos modelos disponibles")

🚀 Lanzando interfaz comparativa...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://98f71e08552ea80962.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---
# 🎓 EJERCICIOS

## 📝 Ejercicio 1: Personalización (BÁSICO)
1. Modifica los ejemplos para incluir más contexto argentino específico
2. Agrega 3 ejemplos nuevos con nombres de barrios porteños
3. Probá con texto de diferentes regiones de Argentina





## 🤔 Preguntas de Reflexión
1. ¿Cuáles son las ventajas y desventajas de cada enfoque?
# **Transformers**

**Ventajas:**

*1)Privacidad y Control*: Los datos se procesan localmente (en tu entorno de Colab, aunque el modelo se descarga), lo que es importante si trabajas con información sensible que no puede salir de tu control.

*2)Costo*: Una vez que el modelo está descargado, el costo de inferencia es nulo o muy bajo (solo el costo computacional de tu entorno). No pagas por cada llamada a una API.

*3)Personalización*: Aunque usamos un modelo pre-entrenado, podrías afinarlo con tus propios datos si necesitaras identificar entidades específicas para tu dominio.

*4)Velocidad*: Para procesamiento por lotes grande, puede ser más rápido que enviar y recibir datos de una API remota, especialmente si tienes acceso a GPU.

**Desventajas:**

*1)Configuración*: Requiere la instalación de librerías (Transformers, PyTorch/TensorFlow) y la descarga del modelo, lo que añade complejidad inicial.

*2)Recursos Computacionales*: Necesitas hardware suficiente (CPU o GPU) para cargar y ejecutar el modelo, especialmente modelos grandes.

*3)Mantenimiento*: Eres responsable de mantener las librerías y el modelo actualizados.

*4)Flexibilidad/Generalidad*: Los modelos pre-entrenados tienen etiquetas fijas (PER, LOC, ORG, MISC en este caso). Si necesitas identificar otros tipos de entidades, tendrías que re-entrenar o usar un modelo diferente.

# **API de Gemini**

**Ventajas:**

*1)Facilidad de Uso*: La integración es muy simple (vimos que solo necesitas una API Key y un par de líneas de código para hacer la llamada). No necesitas preocuparte por la infraestructura o la descarga de modelos grandes.

*2)Modelos Potentes y Actualizados*: Tienes acceso a modelos de lenguaje de vanguardia que Google mantiene y mejora continuamente.

*3)Flexibilidad (Few-shot/Zero-shot)*: Como viste en el prompt que usamos, puedes instruir a Gemini con lenguaje natural para identificar entidades y dar explicaciones. Esto permite adaptarte a nuevos tipos de entidades o formatos de salida sin necesidad de re-entrenar.

*4)Contexto y Explicación*: Gemini puede proporcionar un análisis más contextual y explicaciones sobre la entidad encontrada, algo que un modelo NER clásico no suele hacer por defecto (vimos cómo nos devolvió "[ENTIDAD] → [CATEGORÍA] → [BREVE EXPLICACIÓN]").

**Desventajas:**

*1)Costo*: Generalmente pagas por uso (por cantidad de tokens procesados o llamadas a la API). Esto puede ser costoso para procesar grandes volúmenes de texto.

*2)Latencia*: Hay una latencia asociada a enviar la solicitud a través de la red y esperar la respuesta.

*3)Privacidad*: La información se envía a través de internet a un servicio de terceros. Debes asegurarte de cumplir con las políticas de uso y privacidad, especialmente si trabajas con datos confidenciales.

*4)Dependencia del Proveedor*: Dependes de la disponibilidad y las condiciones del servicio de la API.


# **2. ¿En qué casos usarías un modelo local vs una API?**

**Usar Modelo Local (Transformers):**

Cuando la privacidad de los datos es crítica.
Cuando necesitas procesar grandes volúmenes de texto de forma continua y el costo por llamada a la API es prohibitivo.
Cuando requieres baja latencia y el procesamiento debe ser muy rápido.
Cuando necesitas control total sobre el modelo y la infraestructura.
Si tienes recursos computacionales (especialmente GPU) disponibles.
Si necesitas afinar el modelo con datos específicos de tu dominio.

**Usar API (Gemini):**

Cuando la facilidad de implementación y la rapidez para obtener resultados son prioritarias.
Para prototipado rápido y experimentación con diferentes tareas de PLN.
Cuando necesitas flexibilidad para identificar diferentes tipos de entidades o adaptar el formato de salida fácilmente (zero-shot/few-shot learning).
Para análisis que requieren comprensión contextual o explicaciones detalladas.
Si no tienes acceso a hardware potente para ejecutar modelos grandes localmente.
Para volúmenes de procesamiento moderados donde el costo por llamada es aceptable.



# **3. ¿Cómo evaluarías la precisión de los resultados?**

Evaluar la precisión de un modelo NER requiere un conjunto de datos de "verdad fundamental" (ground truth), es decir, texto donde las entidades nombradas ya han sido anotadas manualmente por humanos.

**Los pasos típicos serían:**

1)Crear un conjunto de prueba: Toma un corpus de texto representativo de tu dominio y haz que anotadores humanos identifiquen y etiqueten todas las entidades de interés (PER, LOC, ORG, MISC, etc.).

2)Procesar el conjunto de prueba: Ejecuta el modelo NER (Transformers o Gemini) sobre este mismo texto sin las anotaciones.

3)Comparar resultados: Compara las entidades identificadas por el modelo con las anotaciones humanas. Puedes usar métricas como:

  Precision: De las entidades que el modelo identificó, ¿cuántas son correctas?

  Recall: De todas las entidades reales en el texto, ¿cuántas identificó el modelo?

  F1-score: Es la media armónica de Precision y Recall, dando un equilibrio entre ambas. Es la métrica más común para NER.

También podrías evaluar si la categoría asignada es correcta y si los límites (start/end) de la entidad son precisos.

4)Análisis de Errores: Revisa los errores (falsos positivos, falsos negativos, errores de categoría) para entender por qué el modelo falla y dónde se puede mejorar.

En el caso de Gemini, como no te da una "confianza" numérica por entidad como Transformers, la evaluación se basaría más en la comparación directa de las entidades y categorías extraídas con el ground truth.



# **4. ¿Qué consideraciones éticas debemos tener en cuenta?**

El uso de NER, especialmente con datos sensibles, conlleva importantes consideraciones éticas:

**1)Privacidad**: Procesar nombres de personas (PER) o ubicaciones específicas (LOC) puede implicar manejar información personal identificable (PII). Debes asegurarte de cumplir con las regulaciones de protección de datos (como GDPR o leyes locales) y anonimizar o pseudonimizar los datos si es necesario.

**2)Sesgos**: Los modelos de lenguaje se entrenan con grandes cantidades de texto y pueden heredar y perpetuar sesgos presentes en esos datos (por ejemplo, asociar ciertos nombres con ciertas ocupaciones o géneros de forma estereotipada). Esto puede afectar la precisión o ser injusto.

**3)Uso Malintencionado**: La capacidad de extraer PII de forma automatizada podría usarse para vigilancia, creación de perfiles no deseados o desinformación.

**4)Transparencia y Explicabilidad**: Con modelos más complejos como Gemini, puede ser difícil entender por qué identificó una entidad de cierta manera. La falta de transparencia puede ser un problema en aplicaciones críticas.

**5)Responsabilidad**: ¿Quién es responsable si el sistema NER comete un error con consecuencias negativas (por ejemplo, omitir una entidad crítica en un informe médico)?



# **5. ¿Cómo escalarías esta solución para procesar miles de documentos?**

Escalar la solución dependería del enfoque elegido:

**Para el enfoque de Transformers (modelo local):**

Procesamiento Distribuido:

Paralelismo: Si tienes muchos documentos, puedes dividirlos y procesarlos en paralelo en múltiples máquinas o núcleos de CPU/GPU.

Frameworks: Usar frameworks de procesamiento distribuido como Apache Spark o Dask para manejar grandes volúmenes de datos y coordinar el procesamiento en un cluster.

Hardware: Utilizar máquinas con más CPU, más RAM o múltiples GPU de alto rendimiento.

Optimización del Código: Asegurarse de que el código de procesamiento sea eficiente y aproveche al máximo el hardware disponible (por ejemplo, procesar en lotes (batches) en la GPU).

Despliegue: Desplegar el modelo en un entorno escalable, como un cluster de Kubernetes, un servicio de serverless computing (AWS Lambda, Google Cloud Functions) con provisionamiento adecuado, o una plataforma de MLOps que maneje el escalado automático basado en la carga.

**Para el enfoque de API (Gemini):**

Aumentar Cuota/Límites: Contactar al proveedor de la API (Google) para aumentar los límites de llamadas por minuto/hora si tu volumen de procesamiento excede los límites estándar.

Procesamiento Asíncrono: Enviar solicitudes a la API de forma asíncrona (en lugar de esperar cada respuesta antes de enviar la siguiente) para maximizar el rendimiento y reducir el tiempo total.

Procesamiento por Lotes (si la API lo soporta): Algunas APIs permiten enviar múltiples textos en una sola solicitud, lo que reduce la latencia y el número total de llamadas.

Manejo de Errores y Reintentos: Implementar una lógica robusta para manejar errores de red o de la API y reintentar las llamadas fallidas.

Monitoreo de Costos: Implementar monitoreo continuo de los costos de la API para asegurarte de que se mantengan dentro del presupuesto.

En resumen, escalar Transformers implica escalar tu propia infraestructura computacional y optimizar el código para el procesamiento local, mientras que escalar con una API implica gestionar el uso de la API, los costos y la comunicación eficiente con el servicio remoto.

## 🔧 Ejercicio 2: Análisis Comparativo (INTERMEDIO)
1. Crea una función que cuente cuántas entidades encuentra cada modelo
2. Implementa un sistema de métricas de tiempo de procesamiento
3. Analiza en qué casos cada modelo funciona mejor

In [16]:
# 📝 ESPACIO PARA TUS EJERCICIOS
# Usa esta celda para experimentar y desarrollar tus soluciones

# Ejemplo: Función para contar entidades por tipo
def contar_entidades_por_tipo(texto):
    """Cuenta entidades por categoría usando Transformers"""
    if not ner_pipeline:
        return {}

    entidades = ner_pipeline(texto)
    conteo = {}

    for ent in entidades:
        tipo = ent['entity_group']
        if tipo in conteo:
            conteo[tipo] += 1
        else:
            conteo[tipo] = 1

    return conteo

# Probar la función
texto_prueba = "Juan Pérez trabaja en Google Argentina en Buenos Aires con María López."
print("📊 Conteo de entidades:")
print(contar_entidades_por_tipo(texto_prueba))

# TODO: Agrega aquí tus propias funciones y experimentos

📊 Conteo de entidades:
{'PER': 2, 'ORG': 1, 'LOC': 1}


---
# 🎯 Conclusión

¡Felicitaciones! Completaste el ejercicio de Reconocimiento de Entidades Nombradas.

## 📚 Lo que aprendiste:
- ✅ Implementar NER con modelos pre-entrenados
- ✅ Usar APIs de IA generativa para tareas de PLN
- ✅ Crear interfaces interactivas con Gradio
- ✅ Comparar diferentes enfoques de NER

## 🔄 Próximos pasos:
1. Experimentá con otros modelos de Hugging Face
2. Probá con textos de diferentes dominios
3. Implementa tu proyecto integrador
4. Compartí tus resultados con la clase

## 📖 Recursos adicionales:
- [Hugging Face Models](https://huggingface.co/models?pipeline_tag=token-classification&language=es)
- [Gradio Documentation](https://gradio.app/docs/)
- [Google AI Studio](https://ai.google.dev/)

---
**¡Éxito en tu trabajo integrador!** 🎓🚀